In [2]:
from coffea.util import load
import os
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea import hist, processor
from pocket_coffea.parameters import defaults

from hist import Hist

import sys


matplotlib.rcParams["figure.dpi"] = 300


/t3home/mmalucch/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/util.py:154: FutureWarning: In coffea version v2023.3.0 (target date: 31 Mar 2023), this will be an error.
(Set coffea.deprecations_as_errors = True to get a stack trace now.)
ImportError: coffea.hist is deprecated
  warnings.warn(message, FutureWarning)


In [14]:

new=False

local = ""  # "/run/user/1000/gvfs/sftp:host=t3ui02.psi.ch,user=mmalucch/"
prefix = local + str(
    os.environ.get("HH", "/pnfs/psi.ch/cms/trivcat/store/user/mmalucch/out_HH4b/")
)

if new:
    prefix="../configs/HH4b/"
    input = prefix + "out_parton_matching_firstB_fulldataset/output_all.coffea"
    o = load(f"{input}")
    col = o["columns"]["GluGlutoHHto4B_poisson"][
        "GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_poisson_kl-1p00_kt-1p00_c2-0p00_2018"
    ]["baseline"]
    print(o["cutflow"])
else:
    prefix="../configs/HH4b/"
    input = prefix + "/out_parton_matching_firstB_fulldataset/output_all.coffea"
    o = load(f"{input}")
    col = o["columns"]["GluGlutoHHto4B"][
        "GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_kl-1p00_kt-1p00_c2-0p00_2018"
    ]["jet_btag_medium"]
    print(o["cutflow"])

for k in col.keys():
    col[k] = col[k].value[col[k].value != -999]

RuntimeError: LZ4F_decompress failed with code: ERROR_decompressionFailed

In [4]:
if new:
    print("\nparton")
    parton_eta=col["bQuark_eta"]
    print(len(parton_eta))
    print(len(parton_eta[abs(parton_eta)>2.5]))
print(len(col["bQuarkHiggsMatched_N"]))
print("\nparton matched")
parton_matched_eta = col["bQuarkHiggsMatched_eta"]
print(len(parton_matched_eta))
print(len(parton_matched_eta[abs(parton_matched_eta) > 2.5]))

if new:
    print("\njet")
    jet_flav = col["JetGoodHiggs_hadronFlavour"]
    # plt.hist(jet_flav)
    print(len(jet_flav))
    print(len(jet_flav[jet_flav != 5]))

    print("\njet macthed")
    jet_matched_flav = col["JetGoodHiggsMatched_hadronFlavour"]
    jet_matched_eta = col["JetGoodHiggsMatched_eta"]
    print(len(jet_matched_flav))
    print(len(jet_matched_flav[(jet_matched_flav != 5)]))

    print("\nout acceptance")
    print(len(jet_matched_flav[(abs(parton_matched_eta) > 2.5) | (jet_matched_flav != 5)]))
    print((jet_matched_flav[(abs(parton_matched_eta) > 2.5) | (jet_matched_flav != 5)]))
    print((jet_matched_eta[(abs(parton_matched_eta) > 2.5) | (jet_matched_flav != 5)]))
    print((parton_matched_eta[(abs(parton_matched_eta) > 2.5) | (jet_matched_flav != 5)]))


    match_eff=len(parton_matched_eta)/len(parton_eta[abs(parton_eta)<2.5])
    print(match_eff)



parton
32704
73
8176

parton matched
31775
10

jet
32704
508

jet macthed
31775
14

out acceptance
24
[4. 4. 5. 5. 5. 4. 4. 0. 4. 5. 0. 0. 4. 0. 5. 4. 0. 5. 5. 5. 5. 0. 4. 5.]
[-1.30859375 -1.72827148 -2.4765625  -2.45458984  2.47949219  0.94946289
 -1.84204102  0.28967285  0.7557373   2.43505859 -0.02117538 -0.04892731
 -1.94165039 -1.33740234  2.20507812  0.31951904 -0.37780762 -2.48681641
 -2.49609375 -2.49511719  2.47460938  0.24578857 -0.79455566 -2.48925781]
[-1.34765625 -1.78515625 -2.515625   -2.515625    2.515625    0.90625
 -1.46484375 -0.03137207  0.66992188  2.5703125   0.15185547 -0.30957031
 -1.89453125 -1.23046875  2.5234375   0.15722656 -0.19042969 -2.5078125
 -2.5390625  -2.5390625   2.5625      0.31738281 -0.59765625 -2.5078125 ]
0.9740359266752499


root dataset

In [5]:
filename ="root://storage01.lcg.cscs.ch:1096//pnfs/lcg.cscs.ch/cms/trivcat//store/mc/Run3Summer22EENanoAODv12/GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_powheg-pythia8/NANOAODSIM/Poisson60KeepRAW_130X_mcRun3_2022_realistic_postEE_v6-v2/50000/fc603037-ef65-4bbf-9cef-934ecec40bbe.root"
filename ="~/hh4b_12.root"
events = NanoEventsFactory.from_root(filename, schemaclass=NanoAODSchema).events()
print("Events read:", len(events))

Events read: 57232


In [6]:
# add column with the index
events.GenPart=ak.with_field(events.GenPart, ak.local_index(events.GenPart, axis=1), "index")
print(events.GenPart.index[0])
print(events.GenPart.index[1])
# get last bquark copy
isB = abs(events.GenPart.pdgId) == 5
isLast = events.GenPart.hasFlags(["isLastCopy"])
isHard = events.GenPart.hasFlags(["fromHardProcess"])
bquarks_last = events.GenPart[isB & isLast & isHard]
# print(bquarks_last.pdgId, bquarks_last.genPartIdxMother)
b_quarks = bquarks_last
while True:
    print("\nloop")
    b_mother = events.GenPart[b_quarks.genPartIdxMother]
    mask_mother=(abs(b_mother.pdgId) == 5) | (abs(b_mother.pdgId) == 25)
    b_quarks=b_quarks[mask_mother]
    bquarks_last=bquarks_last[mask_mother]
    b_mother = b_mother[mask_mother]
    # print("old: ", "pdg", b_quarks.pdgId, "mother_idx",b_quarks.genPartIdxMother, "pt", b_quarks.pt, "index", b_quarks.index)
    # print("mother: ", "pdg", b_mother.pdgId, "mother_idx",b_mother.genPartIdxMother, "pt", b_mother.pt)
    # break
    if ak.all(abs(b_mother.pdgId) == 25):
        break
    b_quarks = ak.where(abs(b_mother.pdgId) == 5, b_mother, b_quarks)
    # print("new: ", "pdg", b_quarks.pdgId, "mother_idx",b_quarks.genPartIdxMother, "pt", b_quarks.pt)

    # print(ak.any(abs(b_mother.pdgId) != 25, axis=1))

print(b_quarks.pdgId, b_quarks.genPartIdxMother, b_quarks.pt)
print(b_quarks.genPartIdxMother, len(b_quarks.genPartIdxMother))

# order higgs pt
higgs = events.GenPart[
            (events.GenPart.pdgId == 25)
            & events.GenPart.hasFlags(["fromHardProcess"])
            & events.GenPart.hasFlags(["isLastCopy"])
        ]
higgs = higgs[ak.num(higgs.childrenIdxG, axis=2) == 2]
print(higgs.pt, higgs.childrenIdxG, higgs.children.pdgId, higgs.children.pt)
print(ak.argsort(higgs.pt,ascending=False))
print(higgs.index)
higgs=higgs[ak.argsort(higgs.pt,ascending=False)]
print(higgs.pt, higgs.childrenIdxG, higgs.children.pdgId, higgs.children.pt)
print(higgs.index)

provenance = ak.where(b_quarks.genPartIdxMother == higgs.index[:,0], 1, 2)
print(provenance)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, ... 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, ... 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65]

loop

loop

loop

loop

loop
[[5, -5, 5, -5], [-5, 5, 5, -5], [-5, 5, ... 5, -5], [-5, 5, 5, -5], [5, 5, -5, -5]] [[15, 15, 16, 16], [14, 14, 15, 15], [16, ... [21, 21, 20, 20], [15, 14, 14, 15]] [[128, 164, 218, 21.7], [93, 44, 105, 31.5, ... 41.1, 57, 101], [168, 59.2, 200, 43]]
[[15, 15, 16, 16], [14, 14, 15, 15], [16, ... [21, 21, 20, 20], [15, 14, 14, 15]] 57232
[[265, 209], [87.2, 108], [162, 72.8], ... [128, 45.6], [114, 100], [254, 178]] [[[17, 18], [19, 20]], [[87, ... 3692789]], [[3692846, 3692847], [3692848, 3692849]]] [[[5, -5], [5, -5]], [[5, -5], [5, -5]], ... [[5, -5], [5, -5]], [[5, -5], [5, -5]]] [[[128, 164], [218, 21.7]], [[44, 93], ... [41.1, 94.8]], [[59.2, 200], [168, 43]]]
[[0, 1], [1, 0], [0, 1], [1, 0], [0, 1], ... [1, 0], [0, 1], [0, 1], [0, 1], [0, 1]]
[[15, 16], [14, 15], [15, 16], [13, 14], ..

In [7]:
# provenance=ak.flatten(provenance)
b_quarks=ak.with_field(b_quarks, provenance, "provenance")
print(b_quarks.provenance)
# plt.hist(b_quarks.eta)
# b_quarks=ak.flatten(b_quarks)
eta=b_quarks.eta[abs(b_quarks.eta)>2.5]
print(len(b_quarks))
print(len(eta))
# plt.hist(eta, bins=50)

[[1, 1, 2, 2], [2, 2, 1, 1], [2, 2, 1, 1, ... 2, 1, 1], [2, 2, 1, 1], [2, 1, 1, 2]]
57232
57232
